<a href="https://colab.research.google.com/github/BinduParvati7/Capstone5588/blob/Hemanth/Group5_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Importing the Required libraries
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import math
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
train = pd.read_csv("/content/drive/MyDrive/DS_Capstone/fraudTrain.csv", index_col=0)
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
test = pd.read_csv("/content/drive/MyDrive/DS_Capstone/fraudTest.csv", index_col=0)
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
merged_df = pd.concat([train, test], axis=0, ignore_index=True)

In [7]:
merged_df = merged_df.drop(['first','last','trans_num'], axis=1)

In [ ]:
merged_df.shape

(1852394, 19)

In [ ]:
#merged_df.to_csv('merged_data.csv', index=False)

In [ ]:
merged_df.head(5)

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,561 Perry Cove,Moravian Falls,NC,28654,36.08,-81.18,3495,"Psychologist, counselling",1988-03-09,1325376018,36.01,-82.05,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.89,-118.21,149,Special educational needs teacher,1978-06-21,1325376044,49.16,-118.19,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,594 White Dale Suite 530,Malad City,ID,83252,42.18,-112.26,4154,Nature conservation officer,1962-01-19,1325376051,43.15,-112.15,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.23,-112.11,1939,Patent attorney,1967-01-12,1325376076,47.03,-112.56,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,408 Bradley Rest,Doe Hill,VA,24433,38.42,-79.46,99,Dance movement psychotherapist,1986-03-28,1325376186,38.67,-78.63,0


In [8]:
merged_df['trans_date_trans_time'] = pd.to_datetime(merged_df['trans_date_trans_time'])
merged_df['dob'] = pd.to_datetime(merged_df['dob'])

In [9]:
# Extract relevant information from datetime columns
merged_df['trans_year'] = merged_df['trans_date_trans_time'].dt.year
merged_df['trans_month'] = merged_df['trans_date_trans_time'].dt.month
merged_df['trans_day'] = merged_df['trans_date_trans_time'].dt.day
merged_df['dob_year'] = merged_df['dob'].dt.year
merged_df['dob_month'] = merged_df['dob'].dt.month
merged_df['dob_day'] = merged_df['dob'].dt.day

# Drop the original datetime columns
merged_df = merged_df.drop(['trans_date_trans_time', 'dob'], axis=1)

In [11]:
# Preprocessing
# Assuming you have already converted 'trans_date_trans_time' and 'dob' columns to datetime
# Encode categorical columns using LabelEncoder
label_encoders = {}
categorical_columns = ['merchant', 'category', 'gender', 'street', 'city', 'state', 'job']
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    merged_df[col] = label_encoders[col].fit_transform(merged_df[col])

# Split the data into features (X) and target (y)
X = merged_df.drop(['is_fraud'], axis=1)
y = merged_df['is_fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 23 columns):
 #   Column       Dtype  
---  ------       -----  
 0   cc_num       int64  
 1   merchant     int64  
 2   category     int64  
 3   amt          float64
 4   gender       int64  
 5   street       int64  
 6   city         int64  
 7   state        int64  
 8   zip          int64  
 9   lat          float64
 10  long         float64
 11  city_pop     int64  
 12  job          int64  
 13  unix_time    int64  
 14  merch_lat    float64
 15  merch_long   float64
 16  is_fraud     int64  
 17  trans_year   int64  
 18  trans_month  int64  
 19  trans_day    int64  
 20  dob_year     int64  
 21  dob_month    int64  
 22  dob_day      int64  
dtypes: float64(5), int64(18)
memory usage: 325.1 MB


**Sequential Model**

Input Layer: It takes an input with the shape of (X_train.shape[1],) which is the shape of your training data.

Dense Layer (64 units): It is a fully connected (dense) layer with 64 units and ReLU activation.

Dense Layer (32 units): Another fully connected layer with 32 units and ReLU activation.

Dense Layer (1 unit): The output layer with a single unit and sigmoid activation. This is a binary classification model, and the sigmoid activation function is often used for binary classification tasks.

After defining the model, you compile it using the Adam optimizer, binary cross-entropy loss (commonly used for binary classification problems), and accuracy as the metric for evaluation.

In [ ]:

# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
37048/37048 [==============================] - 97s 3ms/step - loss: 0.0188 - accuracy: 0.9953 - val_loss: 0.0154 - val_accuracy: 0.9958
Epoch 2/10
37048/37048 [==============================] - 89s 2ms/step - loss: 0.0141 - accuracy: 0.9962 - val_loss: 0.0131 - val_accuracy: 0.9962
Epoch 3/10
37048/37048 [==============================] - 98s 3ms/step - loss: 0.0124 - accuracy: 0.9964 - val_loss: 0.0121 - val_accuracy: 0.9963
Epoch 4/10
37048/37048 [==============================] - 97s 3ms/step - loss: 0.0117 - accuracy: 0.9966 - val_loss: 0.0115 - val_accuracy: 0.9967
Epoch 5/10
37048/37048 [==============================] - 89s 2ms/step - loss: 0.0113 - accuracy: 0.9967 - val_loss: 0.0113 - val_accuracy: 0.9966
Epoch 6/10
37048/37048 [==============================] - 94s 3ms/step - loss: 0.0110 - accuracy: 0.9967 - val_loss: 0.0115 - val_accuracy: 0.9968
Epoch 7/10
37048/37048 [==============================] - 99s 3ms/step - loss: 0.0108 - accuracy: 0.9968 - val_loss: 0

In [ ]:

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

11578/11578 [==============================] - 17s 1ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    368526
           1       0.79      0.57      0.66      1953

    accuracy                           1.00    370479
   macro avg       0.89      0.78      0.83    370479
weighted avg       1.00      1.00      1.00    370479

[[368230    296]
 [   847   1106]]


**Implementing Convolutional Neural Network (CNN)**

This is a sequential model for a 1D Convolutional Neural Network (CNN) designed for some form of sequence data (possibly time-series data). It consists of the following layers:

Conv1D Layer (64 filters, kernel size 3): This layer performs a 1D convolution operation with 64 filters and a kernel size of 3. It uses ReLU activation.

MaxPooling1D Layer (pool size 2): After the convolution, there's a max-pooling layer with a pool size of 2.

Conv1D Layer (128 filters, kernel size 3): Another convolutional layer with 128 filters and kernel size 3, also using ReLU activation.

MaxPooling1D Layer (pool size 2): Another max-pooling layer with a pool size of 2.

Flatten Layer: This layer flattens the output from the previous layers into a 1D vector, preparing it for the fully connected layers.

Dense Layer (64 units): A fully connected layer with 64 units and ReLU activation.

Dropout Layer: A dropout layer with a dropout rate of 0.5, which helps prevent overfitting.

Dense Layer (1 unit): The output layer with a single unit and sigmoid activation, indicating binary classification.

The model is compiled using the Adam optimizer, binary cross-entropy loss, and accuracy as the metric, just like the first model.

In summary, the first model is a feedforward neural network, while the second model is a 1D convolutional neural network, typically used for sequence or time-series data. Both models are designed for binary classification tasks.

In [ ]:
# Define the CNN-based deep learning model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)



Epoch 1/10
37048/37048 [==============================] - 359s 10ms/step - loss: 0.0203 - accuracy: 0.9948 - val_loss: 0.0178 - val_accuracy: 0.9947
Epoch 2/10
37048/37048 [==============================] - 308s 8ms/step - loss: 0.0176 - accuracy: 0.9948 - val_loss: 0.0172 - val_accuracy: 0.9947
Epoch 3/10
37048/37048 [==============================] - 254s 7ms/step - loss: 0.0169 - accuracy: 0.9951 - val_loss: 0.0159 - val_accuracy: 0.9950
Epoch 4/10
37048/37048 [==============================] - 249s 7ms/step - loss: 0.0164 - accuracy: 0.9956 - val_loss: 0.0152 - val_accuracy: 0.9959
Epoch 5/10
37048/37048 [==============================] - 247s 7ms/step - loss: 0.0163 - accuracy: 0.9957 - val_loss: 0.0150 - val_accuracy: 0.9960
Epoch 6/10
37048/37048 [==============================] - 247s 7ms/step - loss: 0.0161 - accuracy: 0.9957 - val_loss: 0.0167 - val_accuracy: 0.9961
Epoch 7/10
37048/37048 [==============================] - 244s 7ms/step - loss: 0.0159 - accuracy: 0.9960 - val

In [ ]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

11578/11578 [==============================] - 32s 3ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    368526
           1       0.75      0.43      0.54      1953

    accuracy                           1.00    370479
   macro avg       0.87      0.71      0.77    370479
weighted avg       1.00      1.00      1.00    370479

[[368253    273]
 [  1121    832]]


**Implementing LSTM Model**

LSTM is a type of recurrent neural network (RNN) architecture, specifically designed to handle and capture long-range dependencies and temporal patterns in sequential data. LSTMs are often used in tasks such as time series analysis, natural language processing, and any task where the order of data points is significant.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
37048/37048 [==============================] - 554s 15ms/step - loss: 0.0223 - accuracy: 0.9949 - val_loss: 0.0178 - val_accuracy: 0.9947
Epoch 2/10
37048/37048 [==============================] - 525s 14ms/step - loss: 0.0156 - accuracy: 0.9959 - val_loss: 0.0149 - val_accuracy: 0.9962
Epoch 3/10
37048/37048 [==============================] - 502s 14ms/step - loss: 0.0132 - accuracy: 0.9964 - val_los

**Implementing GRU Model**

A GRU is a type of recurrent neural network (RNN) that is designed to capture and model sequential data. It is similar to the more complex LSTM (Long Short-Term Memory) but is computationally more efficient and can be easier to train.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model = Sequential()
model.add(GRU(units=64, input_shape=(X_train.shape[1], 1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


**Implementing Auto encoder model**

An autoencoder is a type of neural network used for dimensionality reduction, feature learning, and anomaly detection. It consists of an encoder network that maps the input data to a lower-dimensional representation and a decoder network that attempts to reconstruct the original input from this representation.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the encoder part of the autoencoder
encoder = Sequential()
encoder.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
encoder.add(Dense(64, activation='relu'))

# Define the decoder part of the autoencoder
decoder = Sequential()
decoder.add(Dense(128, activation='relu', input_shape=(64,)))
decoder.add(Dense(X_train.shape[1], activation='sigmoid'))

# Combine the encoder and decoder to create the autoencoder
autoencoder = Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()
autoencoder.fit(X_train, X_train, epochs=5, batch_size=32, validation_split=0.2)
